# Hyperparameter optimization

In [1]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, ReLU, Activation, Input, Reshape, Flatten, Dense, PReLU, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
import dask
from collections import OrderedDict
from lossycomp.dataLoader import DataGenerator, data_preprocessing, split_data
from lossycomp.utils import lr_log_reduction, correlation_5, calculate_MAE_5, mean_squared_error_5

In [2]:
class Autoencoder_2(HyperModel):
    def __init__(self, time, latitude, longitude, channels, strides = 2):
        self.time = time
        self.latitude = latitude
        self.longitude = longitude
        self.channels = channels
        self.strides = strides
    
    def build(self, hp):
        
        def ResBlock(x, num_filter):
            x_in = x
            x = Conv3D(num_filter, (3, 3, 3), strides = (1, 1, 1), padding="same", kernel_regularizer= tf.keras.regularizers.l2(hp6))(x)
            x = ReLU()(x)
            x = Conv3D(num_filter, (3, 3, 3), strides = (1, 1, 1), padding="same", kernel_regularizer= tf.keras.regularizers.l2(hp6))(x)
            x = x_in + x
            x = ReLU()(x)
            return x
    
        inputShape = (self.time, self.latitude, self.longitude, self.channels)
        # Input to the encoder
        inputs = Input(shape=inputShape)
        x = inputs
        
		# Define hyperparameters
		
        hp1 = hp.Int('convs', min_value= 4, max_value= 5, step=1)  # Hyp 1: Number of convolutions [4,5,6]
      
        hp2 = hp.Choice('filter', values=[20, 32, 64]) # Hyp 2: Number of filters.
        
        hp3 = hp.Int('kernel_siz', min_value=3, max_value= 7, step=1) #Hyp 3: Kernel size [3,4,5,6,7]
		
        hp4 = hp.Float('Learning_rate', min_value = 0.0001, max_value = 0.01, sampling = 'log') # Hyp 4: Learning rate (0.01, 0.0001)

        hp5 = hp.Int('num_res_blocks', 0, 3) # Hyp 5: Number of residual blocks [0,1,2,3]
        
        hp6 = hp.Float('l2_reg', min_value = 0.00005 , max_value = 0.5, sampling = 'log' ) # Hyp 6: Number of residual blocks [0,1,2,3]
		
        x = Conv3D(hp2, (hp3, hp3, hp3), strides = (1,1, 1) , padding="same", kernel_regularizer= tf.keras.regularizers.l2(hp6), activation = 'relu')(x)
        
		# Residual Blocks
        for i in range(hp5):
            x = ResBlock(x, hp2)
        
        for i in range(hp1):
            x = Conv3D(hp2, (hp3, hp3, hp3), strides = (self.strides,self.strides,self.strides) , padding="same", kernel_regularizer= tf.keras.regularizers.l2(hp6), activation = 'relu')(x)
        
        volumeSize = K.int_shape(x)
    
        encoder = Model(inputs = inputs, outputs = x, name="Encoder")
      
        # Input to the decoder
        latentInputs = Input(shape=volumeSize[1:])
        
        x = latentInputs
        
        for i in range(hp1-1):
            x = Conv3DTranspose(hp2, (hp3, hp3, hp3), strides = (self.strides,self.strides,self.strides) , padding="same", kernel_regularizer= tf.keras.regularizers.l2(hp6), activation = 'relu')(x)
        
        for i in range(hp5):
            x = ResBlock(x, hp2)
        
        x = Conv3DTranspose(filters = 1, kernel_size = (hp3, hp3, hp3),
                            activation=None, strides = 2 , padding="same", kernel_regularizer= tf.keras.regularizers.l2(hp6))(x)  

        # build the decoder model
        decoder = Model(latentInputs, x, name="Decoder")
        autoencoder = Model(inputs, decoder(encoder(inputs)), name="Autoencoder")
            
        autoencoder.compile(optimizer=keras.optimizers.Adam(hp4),
                      loss=mean_squared_error_5, 
                      metrics=[ tf.keras.losses.MeanSquaredError(), 'MAE']) 
        print(autoencoder.summary())
        return autoencoder

In [3]:
hypermodel = Autoencoder_2(32, 64, 64, 5)

In [4]:
tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=50,
    directory='5_channel',
    project_name='climate_model')

INFO:tensorflow:Reloading Oracle from existing project 5_channel/climate_model/oracle.json
Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 64, 64, 5)]   0         
_________________________________________________________________
Encoder (Functional)         (None, 2, 4, 4, 20)       46000     
_________________________________________________________________
Decoder (Functional)         (None, 32, 64, 64, 1)     33001     
Total params: 79,001
Trainable params: 79,001
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
convs (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 5, 'step': 1, 'sampling': None}
filter (Choice)
{'default': 20, 'conditions': [], 'values': [20, 32, 64], 'ordered': True}
kernel_siz (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 7, 'step': 1, 'sampling': None}
Learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
num_res_blocks (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 3, 'step': 1, 'sampling': None}
l2_reg (Float)
{'default': 5e-05, 'conditions': [], 'min_value': 5e-05, 'max_value': 0.5, 'step': None, 'sampling': 'log'}


In [6]:
dask.config.set(**{'array.slicing.split_large_chunks': False})

file = '/lsdf/kit/scc/projects/abcde/1979/*/ERA5.pl.temperature.nc'
region = "globe"
var = OrderedDict({'t': 1000})

z, mean, std = data_preprocessing(file, var, region)

train, test = split_data(z, 0.70)

leads = dict(time = 32, longitude=64, latitude=64, level=1)

dg_train = DataGenerator(z, 10000, leads, batch_size=10, load=True, mean= mean, std=std, coords = True, standardize = True) 
dg_test = DataGenerator(test, 100, leads, batch_size=10, load=True, mean= mean, std=std, coords = True, standardize = True)

Loading data into RAM
Loading data into RAM


In [ ]:
tuner.search(dg_train, epochs=100,  validation_data=dg_test,  callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
convs             |5                 |?                 
filter            |20                |?                 
kernel_siz        |4                 |?                 
Learning_rate     |0.0022171         |?                 
num_res_blocks    |3                 |?                 
l2_reg            |0.014748          |?                 

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 64, 64, 5)]   0         
_________________________________________________________________
Encoder (Functional)         (None, 1, 2, 2, 20)       199440    
_________________________________________________________________
Decoder (Functional)         (None, 32, 64, 64, 1)     168681    
Total params: 368,121
Trainable params: 368,121
Non-trainable params: 0
__________________